In [11]:
import pandas as pd
import re
from geojson import FeatureCollection, Feature, Point
import json

In [12]:
print('Starting...')
url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.csv'
raw_data = pd.read_csv(url)
print('url fetch correctly')

Starting...
url fetch correctly


In [17]:
time = raw_data['time']
lat = raw_data['latitude']
lon = raw_data['longitude']
depth = raw_data['depth']
mag = raw_data['mag']

zone, place = [], []
for row in raw_data['place']:
    x = re.split(',', row)
    zone.append(x[0])
    place.append(x[-1])

final_data = pd.DataFrame(
{
    "time": time,
    "latitude": lat,
    "longitude": lon,
    "depth": depth,
    "magnitude": mag,
    "zone": zone,
    "place": place
})

In [18]:
final_data.to_json('datamongo.json',orient="records")

In [22]:
df=pd.read_json('datamongo.json').head()
df

,time,latitude,longitude,depth,magnitude,zone,place
0,2021-10-18T17:10:56.914Z,59.692800,-152.328400,73.80,1.80,29 km WSW of Anchor Point,Alaska
1,2021-10-18T16:40:16.700Z,45.368333,-121.701333,2.34,1.08,8 km NNE of Government Camp,Oregon
2,2021-10-18T16:38:44.380Z,35.670667,-117.470667,8.80,1.01,12km SSW of Searles Valley,CA
3,2021-10-18T16:37:01.090Z,46.844333,-121.764833,1.17,0.20,22 km ENE of Ashford,Washington
4,2021-10-18T16:18:55.301Z,-5.503100,147.066700,206.36,4.30,135 km N of Lae,Papua New Guinea


In [28]:
geojson = {'type': 'FeatureCollection', 'features': []}
for _, row in final_data.iterrows():
    feature = {'type': 'Feature',
               'properties': {},
               'geometry': {'type': 'Point',
                            'coordinates': []}}
    feature['geometry']['coordinates'] = [row['longitude'], row['latitude']]
    for prop in final_data.columns:
        feature['properties'][prop] = row[prop]
    geojson['features'].append(feature)
json.dumps(geojson)

'{"type": "FeatureCollection", "features": [{"type": "Feature", "properties": {"time": "2021-10-18T16:38:44.380Z", "latitude": 35.6706667, "longitude": -117.4706667, "depth": 8.8, "magnitude": 1.01, "zone": "12km SSW of Searles Valley", "place": " CA"}, "geometry": {"type": "Point", "coordinates": [-117.4706667, 35.6706667]}}, {"type": "Feature", "properties": {"time": "2021-10-18T16:18:55.301Z", "latitude": -5.5031, "longitude": 147.0667, "depth": 206.36, "magnitude": 4.3, "zone": "135 km N of Lae", "place": " Papua New Guinea"}, "geometry": {"type": "Point", "coordinates": [147.0667, -5.5031]}}, {"type": "Feature", "properties": {"time": "2021-10-18T16:16:30.640Z", "latitude": 44.4998333, "longitude": -110.5523333, "depth": 5.79, "magnitude": 0.76, "zone": "47 km ESE of West Yellowstone", "place": " Montana"}, "geometry": {"type": "Point", "coordinates": [-110.5523333, 44.4998333]}}, {"type": "Feature", "properties": {"time": "2021-10-18T16:13:11.622Z", "latitude": 67.2893, "longitud